In [1]:
using ProgressMeter
using JLD
using Random
include("../alpcahus.jl")
include("../helperFunctions.jl")
include("../cluster.jl")

clusterAssignment (generic function with 1 method)

In [2]:
D = 25 # ambient space dimension
d = 3 # subspace dimension
goodPoints = 6 # good points per subspace
ν1 = 0.1 # noise variance good points
K = 2 # subspace count
varianceRatio = range( 1,300,5) # variance ratio for heatmaps
pointRatio = range(1,50,5) # point ratio between good bad data for heatmaps
trials = 50; # smoother results

In [3]:
seedVector1 = rand(Xoshiro(1234), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector2 = rand(Xoshiro(4321), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector3 = rand(Xoshiro(2143), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector4 = rand(Xoshiro(3412), 1:100000, trials*length(varianceRatio)*length(pointRatio));

In [4]:
trial0Tensor = zeros(length(varianceRatio), length(pointRatio),trials)
trial1Tensor = zeros(length(varianceRatio), length(pointRatio),trials)
trial2Tensor = zeros(length(varianceRatio), length(pointRatio),trials)
trial3Tensor = zeros(length(varianceRatio), length(pointRatio),trials)

count = 1
@showprogress 1 "Running..." for i = 1:length(varianceRatio) # varianceRatio
     for j = 1:length(pointRatio) # prepend Threads.@threads for multi-threading
        for k = 1:trials
            # other parameters
            ν2 = varianceRatio[i]*ν1
            badPoints = round(Int, pointRatio[j]*goodPoints)
            # subspace generation
            U1 = generateSubspace(D, d; seed=seedVector1[count]) 
            U2 = generateSubspace(D, d; seed=seedVector2[count])
            # data generation
            X1 = generateData(U1, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector3[count])
            X2 = generateData(U2, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector4[count])
            # complete data matrix
            X = hcat(X1,X2);
            # cluster assignments
            trueLabels = vec([1*ones(size(X1)[2],1); 2*ones(size(X2)[2],1)]);
            #  trial
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=24, T=3, subspaceMethod=:pca, finalStep=false)
            trial0Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            #  trial
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=24, T=3, subspaceMethod=:pca, finalStep=true)
            trial1Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            #  trail
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=24, T=3, alpcahIter=20, subspaceMethod=:alpcah, finalStep=false, varfloor=1e-9, varianceMethod=:groupless) 
            trial2Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # noisy oracle trial       
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=24, T=3, alpcahIter=20, subspaceMethod=:alpcah, finalStep=true, varfloor=1e-9, varianceMethod=:groupless)
            trial3Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # increase count
            count = count + 1
        end
    end
end

save("trialData.jld", "trial0", trial0Tensor, "trial1", trial1Tensor, "trial2", trial2Tensor, "trial3", trial3Tensor);

Running... 100%|█████████████████████████████████████████| Time: 0:52:45


In [7]:
trialData = load("trialData.jld")
trial0Tensor = trialData["trial0"]
trial1Tensor = trialData["trial1"]
trial2Tensor = trialData["trial2"]
trial3Tensor = trialData["trial3"];

In [17]:
generateHeatmap(pointRatio, varianceRatio, trial0Tensor; plotType=:median, methodType="EKSS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial0_median.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial0_median.pdf"

In [18]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:median, methodType="EKSS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_median.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial1_median.pdf"

In [19]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:median, methodType="ALPCAHUS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_median.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial2_median.pdf"

In [20]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:median, methodType="ALPCAHUS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_median.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial3_median.pdf"

In [21]:
generateHeatmap(pointRatio, varianceRatio, trial0Tensor; plotType=:mean, methodType="EKSS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial0_mean.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial0_mean.pdf"

In [22]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:mean, methodType="EKSS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_mean.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial1_mean.pdf"

In [23]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:mean, methodType="ALPCAHUS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_mean.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial2_mean.pdf"

In [24]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:mean, methodType="ALPCAHUS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_mean.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial3_mean.pdf"

In [25]:
generateHeatmap(pointRatio, varianceRatio, trial0Tensor; plotType=:variance, methodType="EKSS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial0_variance.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial0_variance.pdf"

In [26]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:variance, methodType="EKSS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_variance.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial1_variance.pdf"

In [27]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:variance, methodType="ALPCAHUS (w/o final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_variance.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial2_variance.pdf"

In [28]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:variance, methodType="ALPCAHUS (w/ final assign.)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_variance.pdf")

"/home/javier/Desktop/alpcahus/final_step/trial3_variance.pdf"